- ordem: 

length mínimo: 6

length máximo: 9

- latitude: 

length mínimo: 2

length máximo: 9

- longitude: 

length mínimo: 5

length máximo: 9

- datahora: 

length mínimo: 13

length máximo: 13

- velocidade: 

length mínimo: 1

length máximo: 3

- linha: 

length mínimo: 1

length máximo: 10

- datahoraenvio: 

length mínimo: 13

length máximo: 13

- datahoraservidor: 

length mínimo: 13

length máximo: 13

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer
from sqlalchemy.exc import SQLAlchemyError
import logging

In [2]:
# Definir o diretório base
base_dir = r"C:\Users\wpett\Downloads\Trabalho 3 - Data Mining"

In [3]:
# Configurações do banco de dados PostgreSQL
db_user = ''
db_password = ''
db_host = ''
db_port = ''
db_name = ''
table_name = ''

In [4]:
# Criar a engine de conexão com o PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [5]:
# Configurar o logging para registrar mensagens em um arquivo
logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

In [6]:
# Definir o esquema da tabela
metadata = MetaData()

table = Table(table_name, metadata,
              Column('ordem', String(9)),
              Column('latitude', String(9)),
              Column('longitude', String(9)),
              Column('datahora', String(13)),
              Column('velocidade', String(3)),
              Column('linha', String(10)),
              Column('datahoraenvio', String(13)),
              Column('datahoraservidor', String(13))
             )

# Criar a tabela se ela não existir
metadata.create_all(engine)
logging.info("Tabela criada com sucesso (ou já existia).")

In [7]:
# Função para inserir dados na tabela
def insert_data(file_path, engine, table_name,):
    try:
        # Ler o arquivo JSON
        df = pd.read_json(file_path)
        # Garantir que todos os dados sejam strings
        df = df.astype(str)
        # Inserir os dados no banco de dados
        df.to_sql(table_name, engine, if_exists='append', index=False)
        logging.info(f"[SUCCESS] Dados inseridos no banco de dados a partir de: {file_path}")
    except Exception as e:
        logging.error(f"[FAILURE] Erro ao ler {file_path}: {e}")

In [8]:
# Percorrer o diretório e subdiretórios
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.startswith('2024') and file.endswith('json'):
            file_path = os.path.join(root, file)
            insert_data(file_path, engine, table_name)

# Forçar a coleta de lixo para liberar memória não utilizada
import gc
gc.collect()
